In [1]:
#Import libraries

import numpy as np
import os
import matplotlib.pyplot as plt
import pickle
import math

from keras import layers
from keras import models
from keras import optimizers
from keras import backend as K
from keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2
from keras.utils import to_categorical
from keras.utils import multi_gpu_model
from keras.utils import Sequence
from keras.models import load_model
from tensorflow.distribute import MirroredStrategy

In [2]:
#All paths

Path = "/home/ug-ml/felix-ML/DataGenerator2/Data" #Path where training and validation data is
SaveDataPath = "/home/ug-ml/Documents/GitHub_BigFiles/SaveFolder" #Base directory of place you store information of models
SaveFolderName = "/ConvnetR2" #Will create a folder and put in information about the outcome / inputs
CifFlolder = "/home/ug-ml/felix-ML/DataGenerator2/CifFolder"
ModelName = "/ConvnetR2.hdf5"

#Dont need to change
batch_size = 32

#Can specify which data is used
#LatticeRange = [6, 11]

LatticeRange = [7, 11]





In [3]:
#Load data generators

class FelixSequence(Sequence):
    def __init__(self, x_set, y_set, batch_size, file_type):
        """Here self.x is a list of paths to file_type files. self.y is a
        corresponding list of labels."""
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.file_type = file_type

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return arrs_from_paths(batch_x, self.file_type), np.array(batch_y)

def arrs_from_paths(paths, file_type):
    if file_type == "txt":
        return np.array([np.loadtxt(file_name) for file_name in paths])
    elif file_type == "npy":
        return np.array([np.load(file_name) for file_name in paths]) 
    
    
#Define Data gemerator
rng = np.random.default_rng()

PathOfFile = CifFlolder +"/FilePaths.txt"
with open(PathOfFile) as textFile:
    lines = [line.split() for line in textFile]


training_path = []
training_labels = []
validation_path = []
validation_labels = []
for i in lines:
    PathSplit = i[0].split("/")
    for j in PathSplit:
        if(j == "training"):
            if(float(i[3]) > LatticeRange[0] and float(i[3]) < LatticeRange[1]):
                training_path.append(i[0])
                training_labels.append(int(i[2]) / 9)
                break
            
        elif(j == "validation"):
            if(float(i[3]) > LatticeRange[0] and float(i[3]) < LatticeRange[1]):
                validation_path.append(i[0])
                validation_labels.append(int(i[2]) / 9)
                break

                
trainsize = len(training_path)
validationsize = len(validation_path)

ShuffleTraining = np.arange(trainsize, dtype = np.int)
ShuffleValidation = np.arange(validationsize, dtype = np.int)

training_path_shuffled = training_path.copy()
training_labels_shuffled = training_labels.copy()
validation_path_shuffled = validation_path.copy()
validation_labels_shuffled = validation_labels.copy()

rng.shuffle(ShuffleTraining)
rng.shuffle(ShuffleValidation)

for i in range(0, trainsize):
    training_path_shuffled[i] = training_path[ShuffleTraining[i]]
    training_labels_shuffled[i] = training_labels[ShuffleTraining[i]]
    #print(training_path_shuffled[i], training_labels_shuffled[i])

for i in range(0, validationsize):
    validation_path_shuffled[i] = validation_path[ShuffleValidation[i]]
    validation_labels_shuffled[i] = validation_labels[ShuffleValidation[i]]
    #print(validation_path_shuffled[i], validation_labels_shuffled[i])







print("data done.")
training_seq = FelixSequence(training_path_shuffled, training_labels_shuffled, batch_size, "npy")
print("training_seq done.")
val_images = arrs_from_paths(validation_path_shuffled, "npy")
print("val_images done.")
val_lab = np.array(validation_labels_shuffled.copy())
print("val_lab done.")
print("Complete")

data done.
training_seq done.
val_images done.
val_lab done.
Complete


In [4]:
#Load pretrained model
model = load_model(SaveDataPath + SaveFolderName + ModelName)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d_9 (Separabl (None, 128, 126, 126)     5060      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 128, 63, 63)       0         
_________________________________________________________________
separable_conv2d_10 (Separab (None, 128, 61, 61)       17664     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 128, 30, 30)       0         
_________________________________________________________________
separable_conv2d_11 (Separab (None, 128, 28, 28)       17664     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 128, 14, 14)       0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 25088)            

In [5]:
#Load final 10 output activations into an array of size validation
AllOutputs = np.zeros(validationsize * 1, dtype = np.float32).reshape(validationsize, 1)

#Input 1 image at a time
Input = np.zeros(1 * 36 * 128 * 128, dtype=np.float32).reshape(1, 36, 128, 128)

#Save 1 ourput at a time
Outputs = np.zeros(validationsize * 1, dtype=np.float32).reshape(validationsize)

print("Number of validation data ", validationsize)
for i in range(0, validationsize):
    for layer in model.layers:
        Input[0] = val_images[i]
        keras_function = K.function([model.input], [layer.output])

        A = keras_function([Input, 1])
        if(layer == model.layers[-1]):
            Outputs[i] = A[0][0]
            print(i)
print("Outputs ", Outputs)

#Also make list that CNN predicted
Predicted = model.predict_classes(val_images)

Number of validation data  1150
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
26

In [6]:
length = len(Outputs)
sum = 0
acc = 0
for i in range(0, length):
    print(Outputs[i], val_lab[i])
    predicted = round(9 * Outputs[i])
    if(predicted == round(val_lab[i] * 9)):
        acc+=1
        print("correct")
    sum+=(Outputs[i] - val_lab[i]) ** 2
    
print((sum / length) ** 0.5)
print(acc / length)

0.02210617 0.0
correct
0.9352924 1.0
0.71213037 0.6666666666666666
correct
0.12620291 0.1111111111111111
correct
0.31655535 0.2222222222222222
0.63951564 0.7777777777777778
0.9130664 1.0
0.137739 0.1111111111111111
correct
0.39858463 0.4444444444444444
correct
0.6649886 0.5555555555555556
0.28841737 0.2222222222222222
0.8629416 0.8888888888888888
correct
0.9410144 0.8888888888888888
correct
0.950002 1.0
correct
0.12434259 0.1111111111111111
correct
0.34903887 0.3333333333333333
correct
0.47362024 0.4444444444444444
correct
0.1905165 0.1111111111111111
0.12428719 0.1111111111111111
correct
0.5102584 0.4444444444444444
0.7385308 0.7777777777777778
correct
0.5396213 0.4444444444444444
0.87445605 1.0
0.016789798 0.0
correct
0.7657631 1.0
0.15594007 0.1111111111111111
correct
0.5860606 0.5555555555555556
correct
0.69446 0.6666666666666666
correct
0.21632439 0.2222222222222222
correct
0.3644412 0.3333333333333333
correct
0.784344 1.0
0.35915703 0.3333333333333333
correct
0.5007848 0.44444444

0.13815561 0.1111111111111111
correct
0.9442228 1.0
-0.0037305653 0.0
correct
0.15241925 0.1111111111111111
correct
0.030339561 0.1111111111111111
0.14381602 0.1111111111111111
correct
0.46507353 0.4444444444444444
correct
-0.0073438883 0.0
correct
0.5152323 0.4444444444444444
0.018897638 0.0
correct
0.24969715 0.2222222222222222
correct
0.95547044 1.0
correct
0.62226105 0.6666666666666666
correct
0.7612617 0.7777777777777778
correct
0.11177916 0.1111111111111111
correct
0.7620422 0.7777777777777778
correct
0.8351271 1.0
0.59699863 0.4444444444444444
-0.0041931197 0.0
correct
0.023666091 0.0
correct
0.2432611 0.2222222222222222
correct
0.541654 0.4444444444444444
0.21813795 0.1111111111111111
0.13307273 0.1111111111111111
correct
0.25649676 0.2222222222222222
correct
0.0070043765 0.0
correct
0.20674449 0.2222222222222222
correct
0.005039163 0.0
correct
0.9607462 1.0
correct
0.75377434 0.7777777777777778
correct
0.008889578 0.0
correct
0.46971592 0.4444444444444444
correct
0.8691284 0.7

0.41514847 0.3333333333333333
0.7023821 0.4444444444444444
0.12365024 0.1111111111111111
correct
0.017503817 0.0
correct
0.15816212 0.1111111111111111
correct
0.5614988 0.4444444444444444
0.96549195 1.0
correct
0.5927478 0.5555555555555556
correct
0.73268056 0.6666666666666666
0.12954557 0.1111111111111111
correct
0.66461194 0.6666666666666666
correct
0.71751595 0.8888888888888888
0.1657713 0.1111111111111111
correct
0.7606744 0.8888888888888888
0.81861913 0.8888888888888888
0.35581928 0.3333333333333333
correct
0.57433385 0.5555555555555556
correct
0.99806213 1.0
correct
0.32896668 0.3333333333333333
correct
0.6281032 0.5555555555555556
0.3938287 0.3333333333333333
0.24065673 0.2222222222222222
correct
0.5622028 0.5555555555555556
correct
0.7541081 0.6666666666666666
0.6922142 0.6666666666666666
correct
0.7786827 0.7777777777777778
correct
0.7806367 0.7777777777777778
correct
0.6631636 0.7777777777777778
0.46408078 0.4444444444444444
correct
1.0148797 0.8888888888888888
0.30430475 0.2